This is a homework project in Deep Learning a gyakorlatban Python és Lua alapokon.  
The team members are:

* Gergő László Gyulai
* Ágoston Horváth
* Dávid Frink

You can read more information about our chosen homework at the link below:  
https://www.kaggle.com/competitions/isic-2024-challenge

## Setup

Installing the Kaggle library for downloading datasets from Kaggle

In [ ]:
!pip install kaggle==1.5.12

Configuring Kaggle to access the API

**Warning!** You need to copy your own kaggle.json file into Colab in order to valide yourself

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Download the dataset from the specified Kaggle competition


In [ ]:
!kaggle competitions download -c isic-2024-challenge

Extracting the downloaded dataset

**Warning!** This might take a few minuttes

In [ ]:
!unzip isic-2024-challenge.zip

isic-2024-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  isic-2024-challenge.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Importing libraries

In [ ]:
import pandas as pd
import numpy as np

## Load and prepare sample metadata



In [ ]:
train_df = pd.read_csv('train-metadata.csv')
display(train_df.head(2))

test_df = pd.read_csv('test-metadata.csv')
display(test_df)

Classes distribution: 0: benign, 1: malignant.


In [ ]:
print("\nClass Distribution (%):")
display(train_df['target'].value_counts(normalize=True) *  100)

print("\nClass Distribution:")
display(train_df['target'].value_counts())

Creating a subset of the main dataset for improved handling and processing while maintaining the same distribution.

In [ ]:
benign_df = train_df.query('target == 0')
malignant_df = train_df.query('target == 1')

class_counts = train_df['target'].value_counts(normalize=True)

benign_percentage, malignant_percentage = class_counts

SAMPLE_SIZE = 2500

benign_sample_train_df = benign_df.sample(n=int(benign_percentage * SAMPLE_SIZE), random_state=42)
malignant_sample_train_df = malignant_df.sample(n=int(malignant_percentage * SAMPLE_SIZE), random_state=42)

sample_train_df = pd.concat([benign_sample_train_df, malignant_sample_train_df])

print("\nClass Distribution after sampling (%):")
display(sample_train_df['target'].value_counts(normalize=True) * 100)

print("\nClass Distribution:")
display(sample_train_df['target'].value_counts())

Selecting essential columns in terms of the desired training dataset.

In [ ]:
COLUMNS = [
    'isic_id',
    'target',
    'tbp_lv_nevi_confidence',
    'clin_size_long_diam_mm',
    'tbp_lv_areaMM2',
    'tbp_lv_area_perim_ratio',
    'tbp_lv_color_std_mean',
    'tbp_lv_deltaLBnorm',
    'tbp_lv_minorAxisMM',
    'tbp_lv_perimeterMM'
]

relevant_sample_train_df = sample_train_df[COLUMNS]
relevant_test_df = test_df[COLUMNS[:1] + COLUMNS[2:]]

print("\nSample train dataset: ")
display(relevant_sample_train_df.head(2))

print("\nTest dataset: ")
display(relevant_test_df.head(2))

Saving the existing dataframes for further use.

In [ ]:
relevant_sample_train_df.to_csv('sample-train-metadata.csv')
relevant_test_df.to_csv('sample-test_metadata.csv')

In [ ]:
train = relevant_sample_train_df.to_numpy()
test = relevant_test_df.to_numpy()

train = train[:, 1:]
test = test[:, 1:]

## Loading the images

Handling the absolutely usual .h5py file format, that I have totally seen before in my life...  
(I am just "panic coding" this with Gemini, pls help!)

In [ ]:
import h5py

sample_images = []
sample_keys = ['ISIC_0015670', 'ISIC_3952806', 'ISIC_4803122', 'ISIC_5084643']

file_path = 'train-image.hdf5'
with h5py.File(file_path, 'r') as f:
  for key in sample_keys:
    sample_images.append(f[key][()])


for i, image in enumerate(sample_images):
    print(f"Image {sample_keys[i]} shape: {image.shape}")

Lets see our sample images!

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import io
import numpy as np

fig, axes = plt.subplots(2, 2, figsize=(10, 10))  # Create a 2x2 grid of subplots
axes = axes.flatten()  # Flatten the axes array for easier iteration

for i, image in enumerate(sample_images):
    img = Image.open(io.BytesIO(image))
    img = np.array(img)
    axes[i].imshow(img)  # Display the image on the current subplot
    axes[i].set_title(sample_keys[i])  # Set the title of the subplot

plt.tight_layout()  # Adjust subplot parameters for a tight layout
plt.show()  # Display the plot